## Importing necessary libraries ##

In [ ]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
import joblib
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer, OneHotEncoder, OrdinalEncoder
import sklearn
import xgboost
from joblib import dump
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_scor
from xgboost import XGBRegressor

In [ ]:
class DataProcessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = MinMaxScaler()
        self.standard_scaler = StandardScaler()
        self.yeo_johnson = PowerTransformer(method='yeo-johnson', standardize=True)
        self.one_hot_encoder = None
        self.ordinal_encoder = None
        self.loo_encodings = {}
        self.median_visibility = None
        self.item_weight_medians = {}
        self.outlet_size_modes = {}

    def fit(self, X, y=None):
        # Handle missing Item_Weight
        self.item_weight_medians = X.groupby('Item_Type')['Item_Weight'].median().to_dict()

        # Handle missing Outlet_Size
        self.outlet_size_modes = X.groupby('Outlet_Type')['Outlet_Size'].agg(
            lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown'
        ).to_dict()

        # Replace zero visibility with median visibility
        non_zero_visibility = X.loc[X['Item_Visibility'] > 0, 'Item_Visibility']
        self.median_visibility = non_zero_visibility.median()

        # Fit scalers for numerical columns
        self.scaler.fit(X[['Item_MRP']])
        self.standard_scaler.fit(X[['Item_Weight', 'Item_Visibility']])

        # Fit Yeo-Johnson for skewed features
        skewed_columns = ['Item_Weight', 'Item_Visibility', 'Item_MRP']
        self.yeo_johnson.fit(X[skewed_columns])

        # Fit one-hot encoder for low-cardinality categorical features
        low_cardinality_features = ['Outlet_Type']
        self.one_hot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        self.one_hot_encoder.fit(X[low_cardinality_features])

        # Fit ordinal encoder for features with hierarchy
        ordinal_features = ['Outlet_Size', 'Outlet_Location_Type']
        self.ordinal_encoder = OrdinalEncoder(categories=[
            ['Small', 'Medium', 'High'],  # Outlet_Size
            ['Tier 3', 'Tier 2', 'Tier 1']  # Outlet_Location_Type
        ], handle_unknown='use_encoded_value', unknown_value=-1)
        self.ordinal_encoder.fit(X[ordinal_features])

        # Calculate LOO encoding for high cardinality features, excluding 'Item_Type'
        if y is not None:
            y = pd.Series(y, index=X.index)
            high_cardinality_features = ['Item_Identifier', 'Outlet_Identifier']
            for feature in high_cardinality_features:
                # Group by the feature and calculate the mean of y for each group, excluding the grouping columns
                loo_encoding = X[feature].map(lambda x: y[X[feature] == x].mean()).to_dict()
                self.loo_encodings[feature] = loo_encoding

        return self

    def transform(self, X, y=None):
        X = X.copy()

        # Check required columns
        required_columns = ['Item_Type', 'Item_Weight', 'Item_Visibility', 'Item_MRP', 
                            'Outlet_Type', 'Outlet_Location_Type', 'Outlet_Size']
        for col in required_columns:
            if col not in X.columns:
                raise KeyError(f"The column '{col}' is missing from the input data.")

        # Handle missing values and preprocessing
        X['Item_Weight'] = X['Item_Weight'].fillna(X['Item_Type'].map(self.item_weight_medians))
        X['Outlet_Size'] = X['Outlet_Size'].fillna(X['Outlet_Type'].map(self.outlet_size_modes))
        X['Item_Visibility'] = X['Item_Visibility'].replace(0, self.median_visibility)

        # Normalize and map Item_Fat_Content
        X['Item_Fat_Content'] = X['Item_Fat_Content'].str.strip().str.lower()
        fat_content_map = {'low fat': 'Low Fat', 'lf': 'Low Fat', 'regular': 'Regular', 'reg': 'Regular'}
        X['Item_Fat_Content'] = X['Item_Fat_Content'].map(fat_content_map)

        # One-hot encode Item_Fat_Content
        one_hot = pd.get_dummies(X['Item_Fat_Content'], prefix='Item_Fat_Content')
        X = pd.concat([X.drop(columns=['Item_Fat_Content']), one_hot], axis=1)

        # Outlier capping
        continuous_columns = ['Item_Weight', 'Item_Visibility', 'Item_MRP']
        z_threshold = 3
        for col in continuous_columns:
            upper_bound = X[col].mean() + z_threshold * X[col].std()
            lower_bound = X[col].mean() - z_threshold * X[col].std()
            X[col] = np.clip(X[col], lower_bound, upper_bound)

        # Apply transformations
        skewed_columns = ['Item_Weight', 'Item_Visibility', 'Item_MRP']
        X[skewed_columns] = self.yeo_johnson.transform(X[skewed_columns])
        X[['Item_Weight', 'Item_Visibility']] = self.standard_scaler.transform(X[['Item_Weight', 'Item_Visibility']])
        X[['Item_MRP']] = self.scaler.transform(X[['Item_MRP']])

        # Encode Outlet_Type
        low_cardinality_features = ['Outlet_Type']
        encoded_features = self.one_hot_encoder.transform(X[low_cardinality_features])
        encoded_features = pd.DataFrame(
            encoded_features,
            columns=self.one_hot_encoder.get_feature_names_out(low_cardinality_features),
            index=X.index
        )
        X = pd.concat([X, encoded_features], axis=1).drop(columns=low_cardinality_features)

        # Encode hierarchical features
        ordinal_features = ['Outlet_Size', 'Outlet_Location_Type']
        X[ordinal_features] = self.ordinal_encoder.transform(X[ordinal_features])

        # One-hot encode 'Item_Type'
        one_hot_item_type = pd.get_dummies(X['Item_Type'], prefix='Item_Type')
        X = pd.concat([X.drop(columns=['Item_Type']), one_hot_item_type], axis=1)

        # LOO encoding for high cardinality, excluding 'Item_Type'
        high_cardinality_features = ['Outlet_Identifier']
        for feature in high_cardinality_features:
            if feature in X.columns:
                X[f'{feature}_LOO'] = X[feature].map(self.loo_encodings.get(feature, {})).fillna(0).astype(float)

        X.drop(columns=high_cardinality_features, inplace=True)
        
        # Add new feature engineering
        X['Outlet_Age'] = 2024 - X['Outlet_Establishment_Year']
        X['Visibility_Percentage'] = X['Item_Visibility'] / (X['Item_Visibility'].sum() + 1e-5)
        X['Price_Per_Weight'] = X['Item_MRP'] / (X['Item_Weight'] + 1e-5)
        X['Visibility_to_MRP_Ratio'] = X['Item_Visibility'] / (X['Item_MRP'] + 1e-5)
        X['Discount_Potential'] = X['Item_MRP'] / (X['Item_Visibility'] + 1e-5)

        # Remove spaces in column names
        X.columns = X.columns.str.replace(' ', '_')
        X.drop(columns=['Item_Identifier','Outlet_Establishment_Year'], inplace=True)

        return X

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

# Example usage:
data = pd.read_csv('Train.csv')
target_variable = 'Item_Outlet_Sales'
X = data.drop(columns=[target_variable])
y = data[target_variable]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

processor = DataProcessor()
X_train = processor.fit_transform(X_train, y_train)
X_test = processor.transform(X_test)

print("Training data missing values:\n", X_train.isnull().sum())
print("Testing data missing values:\n", X_test.isnull().sum())


Training data missing values:
 Item_Weight                        0
Item_Visibility                    0
Item_MRP                           0
Outlet_Size                        0
Outlet_Location_Type               0
Item_Fat_Content_Low_Fat           0
Item_Fat_Content_Regular           0
Outlet_Type_Grocery_Store          0
Outlet_Type_Supermarket_Type1      0
Outlet_Type_Supermarket_Type2      0
Outlet_Type_Supermarket_Type3      0
Item_Type_Baking_Goods             0
Item_Type_Breads                   0
Item_Type_Breakfast                0
Item_Type_Canned                   0
Item_Type_Dairy                    0
Item_Type_Frozen_Foods             0
Item_Type_Fruits_and_Vegetables    0
Item_Type_Hard_Drinks              0
Item_Type_Health_and_Hygiene       0
Item_Type_Household                0
Item_Type_Meat                     0
Item_Type_Others                   0
Item_Type_Seafood                  0
Item_Type_Snack_Foods              0
Item_Type_Soft_Drinks              0
Item_Ty

In [23]:
# Save each part separately after fitting
dump(processor.scaler, 'scaler.pkl')
dump(processor.standard_scaler, 'standard_scaler.pkl')
dump(processor.yeo_johnson, 'yeo_johnson.pkl')
dump(processor.one_hot_encoder, 'one_hot_encoder.pkl')
dump(processor.ordinal_encoder, 'ordinal_encoder.pkl')
dump(processor.loo_encodings, 'loo_encodings.pkl')
dump(processor.item_weight_medians, 'item_weight_medians.pkl')
dump(processor.outlet_size_modes, 'outlet_size_modes.pkl')
dump(processor.median_visibility, 'median_visibility.pkl')

['median_visibility.pkl']

In [27]:
from xgboost import XGBRegressor

# Define models with improved hyperparameters
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=200, learning_rate=0.1, random_state=42, verbosity=0),
    'MLP Regressor': MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam',
                                   max_iter=1000, random_state=42)
}

# Train and evaluate models
r2_scores = {'Model': [], 'Training R²': [], 'Testing R²': []}
for model_name, model in models.items():
    print(f"Training {model_name}...")
    
    # Convert X_train to numpy if necessary for XGBoost
    if model_name == 'XGBoost':
        X_train_data, X_test_data = X_train.to_numpy(), X_test.to_numpy()
    else:
        X_train_data, X_test_data = X_train, X_test

    # Fit the model
    model.fit(X_train_data, y_train)
    
    # Predictions
    train_preds = model.predict(X_train_data)
    test_preds = model.predict(X_test_data)
    
    # R² Scores
    r2_scores['Model'].append(model_name)
    r2_scores['Training R²'].append(r2_score(y_train, train_preds))
    r2_scores['Testing R²'].append(r2_score(y_test, test_preds))

# Create a DataFrame for R² scores
r2_scores_df = pd.DataFrame(r2_scores)

# Display results
print("\nModel Performance (R² Scores):")
print(r2_scores_df)

Training Linear Regression...
Training Random Forest...
Training Gradient Boosting...
Training XGBoost...
Training MLP Regressor...

Model Performance (R² Scores):
               Model  Training R²  Testing R²
0  Linear Regression     0.561554    0.577614
1      Random Forest     0.716050    0.604104
2  Gradient Boosting     0.667265    0.601969
3            XGBoost     0.811301    0.569124
4      MLP Regressor     0.521022    0.510415


In [31]:
from sklearn.model_selection import RandomizedSearchCV

# Define XGBoost hyperparameter grid
xgb_param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 1, 5],
    'reg_alpha': [0, 1, 5],
    'reg_lambda': [1, 5, 10],
}

# Initialize XGBRegressor
xgb = XGBRegressor(random_state=42, verbosity=0)

# Perform RandomizedSearchCV
xgb_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=xgb_param_grid,
    n_iter=50,
    scoring='r2',
    cv=5,
    random_state=42,
    n_jobs=-1,
    verbose=2
)

print("Tuning XGBoost...")
xgb_search.fit(X_train, y_train)

# Best XGBoost model
best_xgb = xgb_search.best_estimator_
print(f"Best Params for XGBoost: {xgb_search.best_params_}")
print(f"Best CV R² for XGBoost: {xgb_search.best_score_:.4f}")

# Evaluate XGBoost on train and test sets
xgb_train_r2 = best_xgb.score(X_train, y_train)
xgb_test_r2 = best_xgb.score(X_test, y_test)
print(f"XGBoost Training R²: {xgb_train_r2:.4f}")
print(f"XGBoost Testing R²: {xgb_test_r2:.4f}")

Tuning XGBoost...
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Params for XGBoost: {'subsample': 1.0, 'reg_lambda': 5, 'reg_alpha': 0, 'n_estimators': 500, 'max_depth': 4, 'learning_rate': 0.01, 'gamma': 5, 'colsample_bytree': 0.8}
Best CV R² for XGBoost: 0.5968
XGBoost Training R²: 0.6251
XGBoost Testing R²: 0.6149


In [35]:
import pickle

# Ensure X_train and X_test are NumPy arrays
X_train_np = X_train.to_numpy() if isinstance(X_train, pd.DataFrame) else X_train
X_test_np = X_test.to_numpy() if isinstance(X_test, pd.DataFrame) else X_test

# Ensure y_train and y_test are NumPy arrays
y_train_np = y_train.to_numpy() if isinstance(y_train, pd.Series) else y_train
y_test_np = y_test.to_numpy() if isinstance(y_test, pd.Series) else y_test

# Best Parameters from Hyperparameter Tuning
best_params = {
    'subsample': 1.0,
    'reg_lambda': 5,
    'reg_alpha': 0,
    'n_estimators': 500,
    'max_depth': 4,
    'learning_rate': 0.01,
    'gamma': 5,
    'colsample_bytree': 0.8,
    'random_state': 42,
    'verbosity': 0,
    'enable_categorical': True  # Ensure proper handling if using categorical data
}

# Retrain the XGBoost Model
print("Retraining XGBoost with best parameters...")
best_xgb = XGBRegressor(**best_params)
best_xgb.fit(X_train_np, y_train_np)

# Evaluate on Training and Test Sets
xgb_train_r2 = best_xgb.score(X_train_np, y_train_np)
xgb_test_r2 = best_xgb.score(X_test_np, y_test_np)

print(f"XGBoost Retrained Training R²: {xgb_train_r2:.4f}")
print(f"XGBoost Retrained Testing R²: {xgb_test_r2:.4f}")

# Save the Retrained Model
model_path = 'best_xgb_model.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(best_xgb, f)

print(f"XGBoost model saved to {model_path}")

Retraining XGBoost with best parameters...
XGBoost Retrained Training R²: 0.6251
XGBoost Retrained Testing R²: 0.6149
XGBoost model saved to best_xgb_model.pkl
